In [ ]:
!pip install telebot

In [ ]:
!pip install roboflow supervision opencv-python

In [ ]:
!pip install inference

In [ ]:
# Импортируем необходимые библиотеки
import telebot
import requests
from inference import get_model
import supervision as sv
import cv2

# Импортируем ключи из конфигурационного файла
from config import API_TOKEN, api_key

bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am EchoBot.
I am here to echo your kind words back to you. Just say anything nice and I'll say the exact same thing to you!\
""")

# Handle all other messages with content_type 'photo'
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # Сохраняем полученное фото
    photo = message.photo[-1]
    file_info = bot.get_file(photo.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    save_path = 'photo.jpg'
    with open(save_path, 'wb') as new_file:
        new_file.write(downloaded_file)
        bot.reply_to(message, 'Фотография сохранена.')
    
    # Загружаем фото для обработки
    image_file = "photo.jpg"
    image = cv2.imread(image_file)

    # Выполняем анализ с помощью модели и получаем разметку
    model = get_model(model_id="car-damage-coco-dataset/4", api_key=api_key)  # исправлено здесь
    results = model.infer(image)[0]
    detections = sv.Detections.from_inference(results)
    print(detections)

    # Аннотируем изображение с помощью supervision
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    annotated_image = bounding_box_annotator.annotate(scene=image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

    sv.plot_image(annotated_image)

    # Сохраняем аннотированное изображение
    annotated_image_path = 'annotated_photo.jpg'
    cv2.imwrite(annotated_image_path, annotated_image)

    # Отправляем аннотированное изображение пользователю
    with open(annotated_image_path, 'rb') as photo:
        bot.send_photo(message.chat.id, photo)

    # Отвечаем пользователю
    bot.reply_to(message, 'Фотография с разметкой отправлена.')

# Запускаем бота
bot.infinity_polling()